In [1]:
import numpy as np
import matplotlib.pyplot as plt


## Importar dataset

In [2]:
def carregar_imatges(ruta):
    with open(ruta, 'rb') as f:
        dades = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
    return dades.reshape(-1, 28, 28)

def carregar_etiquetes(ruta):
    with open(ruta, 'rb') as f:
        dades = np.frombuffer(f.read(), dtype=np.uint8, offset=8)
    return dades

X_train = carregar_imatges("train-images.idx3-ubyte")
y_train = carregar_etiquetes("train-labels.idx1-ubyte")
X_test = carregar_imatges("t10k-images.idx3-ubyte")
y_test = carregar_etiquetes("t10k-labels.idx1-ubyte")

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


### Mostrar imatges del dataset

In [32]:

# # Mostrar las primeras 5 imágenes de entrenamiento
# for i in range(5):
#     plt.imshow(X_train[i], cmap='gray')
#     plt.title(f"Etiqueta: {y_train[i]}")
#     plt.show()

In [4]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y

# def backward_prop(N1, Z1, N2, Z2, W1, W2, X, Y):
#     m = Z2.shape[1]
#     one_hot_Y = one_hot(Y)
#     dN2 = 2 * (Z2 - one_hot_Y)
#     dW2 = dN2.dot(Z1.T)/m
#     dB2 = np.sum(dN2,1)/m
#     dN1 = W2.T.dot(N2) * de_Relu(N1)
#     dW1 = dN1.dot(X.T)/m
#     dB1 = np.sum(dN1,1)/m

#     return dW1, dB1, dW2, dB2

def accuracy(predict, expected):
    return np.sum(predict == expected) / expected.size

    
class linear_layer():
    def __init__(self, input_dimensions, output_dimensions):
        self.W = 2 * np.random.rand(input_dimensions, output_dimensions) - 1
        self.B = np.random.rand(1, output_dimensions)

    def forward(self, x):
        self.x = x
        return x @ self.W + self.B
    
    def backward(self, error):
        self.dW = self.x.T @ error
        self.dB = error.sum(axis = 0, keepdims=True)
        return error @ self.W.T

    def update_parameters(self, lr):
        self.W -= lr * self.dW
        self.B -= lr * self.dB


## Definir capes d'activació i input

In [5]:
class Relu_layer():
    def __init__(self):
        pass

    def forward(self, x):
        self.Z = np.maximum(0, x)
        return self.Z
    
    def backward(self, error):
        return error * (self.Z > 0)
    
    def update_parameters(self, ir):
        pass

class input_layer():
    def __init__(self, input_size):
        self.input_size = input_size

    def forward(self, x):
        return x
    
    def backward(self, error):
        return error
    
    def update_parameters(self, ir):
        pass

## Model de xarxa neuronal

In [6]:
class NN_model():
    def __init__(self, input_layer_dimensions, hidden_layer_dimensions, output_layer_dimensions):
        self.sequential = [input_layer(input_layer_dimensions),
                           linear_layer(input_layer_dimensions,hidden_layer_dimensions),
                           Relu_layer(),
                           linear_layer(hidden_layer_dimensions,output_layer_dimensions)]
    
    def forward(self, x):
        for layer in self.sequential:
            y = layer.forward(x)
            x = y
        return y
    
    def backward(self, error):
        for layer in reversed(self.sequential):
            error = layer.backward(error)
        return error
    
    def update_parameters(self, learning_rate=0.03):
        for layer in self.sequential:
            layer.update_parameters(learning_rate)
    
model = NN_model(784, 128, 10)

# y_pred = model.forward(np.array([[1, -10, 1, -8]]))
# print(y_pred)
# y_real = np.array([1,0])
# error = 2 * np.abs(y_pred - y_real)
# model.backward(error)

## Train test loops

In [7]:
def train(model, x_train, y_train, lr=0.03, batch_size=64, epochs=10):
    # Número de files d'x_train
    n_samples = x_train.shape[0]

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        # Barrejar les dades per a millor entrenament
        indexes = np.random.permutation(n_samples)
        x_train = x_train[indexes]
        y_train = y_train[indexes]

        # Entrenament per batches (lotes)
        for i in range(0, n_samples, batch_size):
            x_batch = x_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]

            # Forward
            y_pred = model.forward(x_batch)

            # Calcular l'error
            error = (y_pred - y_batch)

            # Backward
            model.backward(2 * error / batch_size)

            # Actualitzar paràmetres
            model.update_parameters(lr)

        # Evaluar después de cada época
        loss = test(model, x_train, y_train)
        print(f"Loss after epoch {epoch + 1}: {loss}")

def test(model, x_test, y_test):
    # Forward en el conjunto de prueba
    y_pred = model.forward(x_test)

    # Calcular el error cuadrático medio (MSE)
    loss = np.mean((y_pred - y_test) ** 2)
    return loss
    
        

## Dataloader

In [37]:
class dataloader:
    def __init__(self, X, y, batch_size, shuffle=True):
        self.bs = batch_size

        randomizer = np.arange(len(y))

        if shuffle:
            np.random.shuffle(randomizer)
        self.X = X[randomizer]
        self.y = y[randomizer]

        def __len__(self):
            return len(self.y)
        
        def __getitem__(self, idx):
            return self.X[idx] / 255, self.y[idx]
        

In [38]:
# Normalización
X_train = X_train.reshape(X_train.shape[0], -1) / 255.0
X_test = X_test.reshape(X_test.shape[0], -1) / 255.0

y_train = y_train.astype(int)
y_train = one_hot(y_train)

train(model, X_train, y_train, lr=0.03, batch_size=128, epochs=10)

loss = test(model, X_test, y_test)
print(f"Final loss on test set: {loss}")

# # Mostrar las primeras 5 imágenes de entrenamiento
# for i in range(5):
#     plt.imshow(X_train[i], cmap='gray')
#     plt.title(f"Etiqueta: {y_train[i]}")
#     plt.show()


Epoch 1/10
Loss after epoch 1: 9.500820355405874e+101
Epoch 2/10
Loss after epoch 2: 6.101530954254613e+76
Epoch 3/10
Loss after epoch 3: 3.9184700471201326e+51
Epoch 4/10
Loss after epoch 4: 2.5164844078142487e+26
Epoch 5/10
Loss after epoch 5: 16.250985278009676
Epoch 6/10
Loss after epoch 6: 0.0899890681762793
Epoch 7/10
Loss after epoch 7: 0.09000146895684728
Epoch 8/10
Loss after epoch 8: 0.08998746010509717
Epoch 9/10
Loss after epoch 9: 0.08999148401807631
Epoch 10/10
Loss after epoch 10: 0.08998259675734055


ValueError: operands could not be broadcast together with shapes (10000,10) (10000,) 